In [1]:
# start a Spak session
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("final_project_yelp_review_distribution") \
    .getOrCreate()

# get a spark context from the session, and set verbose logging
sc = spark.sparkContext
sc.setLogLevel("ERROR")
sc

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/16 19:46:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[*] appName=final_project_yelp_review_distribution>

In [2]:
df = spark.read\
    .option("header","true")\
    .option("inferSchema","true")\
    .json('shared/student-data/yelp/yelp_academic_dataset_business.json')

df.show(10)
df.printSchema()

+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|       city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|        921 Pearl St|{null, null, 'bee...|6iYb2HFDywm3zjuRg...|Gastropubs, Food,...|    Boulder|{11:0-23:0, 11:0-...|      1|   40.0175444|   -105.2833481| Oskar Blues Taproom|      80302|          86|  4.0|   CO|
| 7000 NE Airport Way|{null, null, u'be...|tCbdrRPZA0oiIYSmH...|Salad, Soup, Sand...|   Portland|{5:0-18:0, 5:0-18...|      1|45.5889058992|

In [3]:
#filter businesses only are restaurants
from pyspark.sql.functions import expr, lit, col
df1=df.filter(col("categories").contains("Restaurants"))

df1.show(10)

+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|       city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|        921 Pearl St|{null, null, 'bee...|6iYb2HFDywm3zjuRg...|Gastropubs, Food,...|    Boulder|{11:0-23:0, 11:0-...|      1|   40.0175444|   -105.2833481| Oskar Blues Taproom|      80302|          86|  4.0|   CO|
| 7000 NE Airport Way|{null, null, u'be...|tCbdrRPZA0oiIYSmH...|Salad, Soup, Sand...|   Portland|{5:0-18:0, 5:0-18...|      1|45.5889058992|

In [4]:
df1.count()

50763

In [ ]:
#from pyspark.sql.functions import expr, lit, col
#df1=df.filter(col("categories").contains("Restaurants")|col("categories").contains("Restaurant"))

#df1.show()

In [ ]:
#df1.count()

In [5]:
from pyspark.sql.functions import asc, desc
df1.select('stars','state','city','review_count')\
   .sort(desc("stars"),asc("state"),asc('city'),desc("review_count"))\
   .show()

+-----+-----+---------------+------------+
|stars|state|           city|review_count|
+-----+-----+---------------+------------+
|  5.0|   BC|   Bowen Island|           8|
|  5.0|   BC|        Burnaby|           7|
|  5.0|   BC|        Burnaby|           7|
|  5.0|   BC|        Burnaby|           6|
|  5.0|   BC|        Burnaby|           5|
|  5.0|   BC|      Coquitlam|          11|
|  5.0|   BC|      Coquitlam|           6|
|  5.0|   BC|          Delta|          11|
|  5.0|   BC|          Delta|           5|
|  5.0|   BC|New Westminster|           7|
|  5.0|   BC|North Vancouver|          20|
|  5.0|   BC|North Vancouver|          18|
|  5.0|   BC|North Vancouver|          11|
|  5.0|   BC|North Vancouver|          10|
|  5.0|   BC|North Vancouver|           7|
|  5.0|   BC|North Vancouver|           6|
|  5.0|   BC|North Vancouver|           5|
|  5.0|   BC|North Vancouver|           5|
|  5.0|   BC| Port Coquitlam|           5|
|  5.0|   BC|     Port Moody|          47|
+-----+----

In [6]:
#Yelp User can choose a specific location(state,city)
#choose the highest stars, then the highest review count
#finally find the perfect restaurant name
from pyspark.sql.functions import asc, desc
df1.select('state','city','stars','review_count','name')\
  .sort(asc("state"),asc('city'),desc('stars'),desc("review_count"))\
  .show()

+-----+------------+-----+------------+--------------------+
|state|        city|stars|review_count|                name|
+-----+------------+-----+------------+--------------------+
|  ABE|   Vancouver|  4.5|          14| Kitanoya Guu Garlic|
|   BC|     BURNABY|  3.5|          59|                IHOP|
|   BC|     BURNABY|  2.5|          31|                IHOP|
|   BC|Bowen Island|  5.0|           8|    Shika Provisions|
|   BC|Bowen Island|  4.5|          26|Cocoa West Chocol...|
|   BC|Bowen Island|  4.5|          13|Barcelona Tapas &...|
|   BC|Bowen Island|  4.0|          52|  Tuscany Restaurant|
|   BC|Bowen Island|  4.0|          40|       The Snug Cafe|
|   BC|Bowen Island|  4.0|          30|Artisan Eats Cafe...|
|   BC|Bowen Island|  4.0|          24|    Bowen Island Pub|
|   BC|Bowen Island|  4.0|          22|     Rustique Bistro|
|   BC|Bowen Island|  4.0|          16|    Miksa Restaurant|
|   BC|Bowen Island|  4.0|           9|     Branch on Bowen|
|   BC|Bowen Island|  4.

In [7]:
# A rating distribution by stars for each state, each city
#count means the number of restaurants which has a specific star in that area
from pyspark.sql.functions import asc, desc
df1.groupBy('stars','state','city').count().sort(desc("stars"),desc('count')).show()

+-----+-----+---------------+-----+
|stars|state|           city|count|
+-----+-----+---------------+-----+
|  5.0|   OR|       Portland|  351|
|  5.0|   TX|         Austin|  255|
|  5.0|   FL|        Orlando|   85|
|  5.0|   GA|        Atlanta|   72|
|  5.0|   BC|      Vancouver|   66|
|  5.0|   OH|       Columbus|   59|
|  5.0|   MA|         Boston|   38|
|  5.0|   OR|      Beaverton|   21|
|  5.0|   WA|      Vancouver|   21|
|  5.0|   CO|        Boulder|   17|
|  5.0|   MA|      Cambridge|   13|
|  5.0|   FL|      Kissimmee|   11|
|  5.0|   GA|        Decatur|    9|
|  5.0|   FL|        Sanford|    9|
|  5.0|   OR|         Tigard|    8|
|  5.0|   BC|North Vancouver|    8|
|  5.0|   MA|          Salem|    8|
|  5.0|   FL|    Winter Park|    8|
|  5.0|   MA|     Somerville|    7|
|  5.0|   FL|          Cocoa|    7|
+-----+-----+---------------+-----+
only showing top 20 rows



In [8]:
# A rating distribution by stars for each state, I'll do a data visualization of this distribution by states by stars
#the data visualization is via d3.js, see code details from the link below
#https://observablehq.com/d/b2caae081096ca21?ui=classic
#count means the number of restaurants which has a specific star in that area
from pyspark.sql.functions import asc, desc
df1.groupBy('stars','state').count().sort(desc("stars"),desc('count')).show()

+-----+-----+-----+
|stars|state|count|
+-----+-----+-----+
|  5.0|   OR|  413|
|  5.0|   TX|  279|
|  5.0|   MA|  193|
|  5.0|   FL|  184|
|  5.0|   GA|  108|
|  5.0|   BC|  100|
|  5.0|   OH|   91|
|  5.0|   CO|   23|
|  5.0|   WA|   21|
|  4.5|   OR| 1581|
|  4.5|   FL| 1242|
|  4.5|   MA| 1157|
|  4.5|   TX| 1016|
|  4.5|   BC|  681|
|  4.5|   GA|  610|
|  4.5|   OH|  575|
|  4.5|   CO|  118|
|  4.5|   WA|  109|
|  4.5|  ABE|    1|
|  4.0|   MA| 2706|
+-----+-----+-----+
only showing top 20 rows



In [9]:
#Business chooses a high-volume location(specific postal_code) to open a restaurant.
#Business can see the higest review count means the specific location for restaurants has a big quantity of customers
#It's a good idea to open a new restaurant in that area.
from pyspark.sql.functions import asc, desc
df1.groupBy('state','city','postal_code').sum('review_count')\
   .withColumnRenamed('sum(review_count)', 'total_review_count')\
   .sort(desc('total_review_count')).show(20)

+-----+---------+-----------+------------------+
|state|     city|postal_code|total_review_count|
+-----+---------+-----------+------------------+
|   TX|   Austin|      78704|            124492|
|   FL|  Orlando|      32819|            121451|
|   TX|   Austin|      78701|            120005|
|   OR| Portland|      97214|            104339|
|   MA|   Boston|      02116|             93835|
|   OR| Portland|      97209|             74662|
|   OR| Portland|      97204|             65349|
|   MA|   Boston|      02113|             64217|
|   TX|   Austin|      78702|             63891|
|   MA|Cambridge|      02138|             58856|
|   MA|Cambridge|      02139|             58794|
|   OR| Portland|      97205|             57987|
|   GA|  Atlanta|      30309|             54481|
|   OR| Portland|      97202|             53466|
|   GA|  Atlanta|      30308|             51272|
|   GA|  Atlanta|      30318|             50933|
|   OH| Columbus|      43215|             49479|
|   FL|  Orlando|   

In [10]:
# sort by stars, find each restuarant's attributes characteristics
from pyspark.sql.functions import asc, desc
df1.select('stars','attributes.BusinessAcceptsCreditCards','attributes.RestaurantsReservations',\
           'attributes.WiFi','attributes.RestaurantsDelivery','attributes.RestaurantsTakeOut',\
          'attributes.BusinessParking','attributes.RestaurantsGoodForGroups',\
          'attributes.HasTV','attributes.RestaurantsPriceRange2',\
          'attributes.GoodForMeal','attributes.OutdoorSeating').sort(desc('stars')).show(20)

+-----+--------------------------+-----------------------+-------+-------------------+------------------+--------------------+------------------------+-----+----------------------+--------------------+--------------+
|stars|BusinessAcceptsCreditCards|RestaurantsReservations|   WiFi|RestaurantsDelivery|RestaurantsTakeOut|     BusinessParking|RestaurantsGoodForGroups|HasTV|RestaurantsPriceRange2|         GoodForMeal|OutdoorSeating|
+-----+--------------------------+-----------------------+-------+-------------------+------------------+--------------------+------------------------+-----+----------------------+--------------------+--------------+
|  5.0|                      True|                  False|   null|               null|              True|{'garage': False,...|                    True| null|                     2|                null|          null|
|  5.0|                      null|                   null|   null|               null|              null|{'garage': False,...|      

In [11]:
#Most 5 star restaurants do accept creditCards
df1.groupBy('stars','attributes.BusinessAcceptsCreditCards').count().sort(desc('stars'),desc('count')).show(5)

+-----+--------------------------+-----+
|stars|BusinessAcceptsCreditCards|count|
+-----+--------------------------+-----+
|  5.0|                      True| 1000|
|  5.0|                      null|  352|
|  5.0|                     False|   60|
|  4.5|                      True| 5621|
|  4.5|                      null| 1181|
+-----+--------------------------+-----+
only showing top 5 rows



In [12]:
#most 5 star restaurants don't need reservations
df1.groupBy('stars','attributes.RestaurantsReservations').count().sort(desc('stars'),desc('count')).show(5)

+-----+-----------------------+-----+
|stars|RestaurantsReservations|count|
+-----+-----------------------+-----+
|  5.0|                  False|  626|
|  5.0|                   null|  598|
|  5.0|                   True|  186|
|  5.0|                   None|    2|
|  4.5|                  False| 4069|
+-----+-----------------------+-----+
only showing top 5 rows



In [13]:
#most 5 star restaurants are WiFi free
df1.groupBy('stars','attributes.WiFi').count().sort(desc('stars'),desc('count')).show(5)

+-----+-------+-----+
|stars|   WiFi|count|
+-----+-------+-----+
|  5.0|   null|  552|
|  5.0|u'free'|  428|
|  5.0|  u'no'|  313|
|  5.0|   'no'|   79|
|  5.0| 'free'|   36|
+-----+-------+-----+
only showing top 5 rows



In [14]:
#most 5 star restaurants provide delivery
df1.groupBy('stars','attributes.RestaurantsDelivery').count().sort(desc('stars'),desc('count')).show(5)

+-----+-------------------+-----+
|stars|RestaurantsDelivery|count|
+-----+-------------------+-----+
|  5.0|               True|  553|
|  5.0|              False|  469|
|  5.0|               null|  318|
|  5.0|               None|   72|
|  4.5|               True| 3198|
+-----+-------------------+-----+
only showing top 5 rows



In [15]:
#most 5 star restaurants provide food takeout
df1.groupBy('stars','attributes.RestaurantsTakeOut').count().sort(desc('stars'),desc('count')).show(5)

+-----+------------------+-----+
|stars|RestaurantsTakeOut|count|
+-----+------------------+-----+
|  5.0|              True| 1063|
|  5.0|              null|  229|
|  5.0|             False|   86|
|  5.0|              None|   34|
|  4.5|              True| 6012|
+-----+------------------+-----+
only showing top 5 rows



In [16]:
##most 5 star restaurants are good for groups
df1.groupBy('stars','attributes.RestaurantsGoodForGroups').count().sort(desc('stars'),desc('count')).show(5)

+-----+------------------------+-----+
|stars|RestaurantsGoodForGroups|count|
+-----+------------------------+-----+
|  5.0|                    null|  861|
|  5.0|                    True|  430|
|  5.0|                   False|  121|
|  4.5|                    True| 4129|
|  4.5|                    null| 1923|
+-----+------------------------+-----+
only showing top 5 rows



In [17]:
#most 5 star restaurants price are in range 1
df1.groupBy('stars','attributes.RestaurantsPriceRange2').count().sort(desc('stars'),desc('count')).show(5)

+-----+----------------------+-----+
|stars|RestaurantsPriceRange2|count|
+-----+----------------------+-----+
|  5.0|                  null|  715|
|  5.0|                     1|  447|
|  5.0|                     2|  225|
|  5.0|                     3|   17|
|  5.0|                     4|    8|
+-----+----------------------+-----+
only showing top 5 rows



In [18]:
#most 5 star restaurants don't have TV
df1.groupBy('stars','attributes.HasTV').count().sort(desc('stars'),desc('count')).show(5)

+-----+-----+-----+
|stars|HasTV|count|
+-----+-----+-----+
|  5.0| null|  568|
|  5.0|False|  523|
|  5.0| True|  321|
|  4.5| True| 3009|
|  4.5|False| 2684|
+-----+-----+-----+
only showing top 5 rows



In [19]:
#most 5 star restaurants have outdoor seating
df1.groupBy('stars','attributes.OutdoorSeating').count().sort(desc('stars'),desc('count')).show(5)

+-----+--------------+-----+
|stars|OutdoorSeating|count|
+-----+--------------+-----+
|  5.0|          True|  575|
|  5.0|          null|  470|
|  5.0|         False|  299|
|  5.0|          None|   68|
|  4.5|          True| 3148|
+-----+--------------+-----+
only showing top 5 rows



In [20]:
#most 5 star restaurants are good for kids
df1.groupBy('stars','attributes.GoodForKids').count().sort(desc('stars'),desc('count')).show(5)

+-----+-----------+-----+
|stars|GoodForKids|count|
+-----+-----------+-----+
|  5.0|       null|  845|
|  5.0|       True|  507|
|  5.0|      False|   60|
|  4.5|       True| 4373|
|  4.5|       null| 2026|
+-----+-----------+-----+
only showing top 5 rows



In [21]:
#most 5 star restaurants noiselevel is average
df1.groupBy('stars','attributes.NoiseLevel').count().sort(desc('stars'),desc('count')).show(5)

+-----+----------+-----+
|stars|NoiseLevel|count|
+-----+----------+-----+
|  5.0|      null| 1015|
|  5.0|u'average'|  191|
|  5.0|  u'quiet'|  136|
|  5.0| 'average'|   44|
|  5.0|   'quiet'|   17|
+-----+----------+-----+
only showing top 5 rows



In [22]:
#most 5 star restaurants don't have table service
df1.groupBy('stars','attributes.RestaurantsTableService').count().sort(desc('stars'),desc('count')).show(5)


+-----+-----------------------+-----+
|stars|RestaurantsTableService|count|
+-----+-----------------------+-----+
|  5.0|                   null|  768|
|  5.0|                  False|  466|
|  5.0|                   True|  178|
|  4.5|                   null| 3320|
|  4.5|                  False| 1986|
+-----+-----------------------+-----+
only showing top 5 rows



In [23]:
df1.groupBy('stars','attributes.NoiseLevel').count().sort(desc('stars'),desc('count')).show(5)

+-----+----------+-----+
|stars|NoiseLevel|count|
+-----+----------+-----+
|  5.0|      null| 1015|
|  5.0|u'average'|  191|
|  5.0|  u'quiet'|  136|
|  5.0| 'average'|   44|
|  5.0|   'quiet'|   17|
+-----+----------+-----+
only showing top 5 rows



In [24]:
df1.groupBy('stars','attributes.Alcohol').count().sort(desc('stars'),desc('count')).show(5)

+-----+----------------+-----+
|stars|         Alcohol|count|
+-----+----------------+-----+
|  5.0|            null|  597|
|  5.0|         u'none'|  494|
|  5.0|u'beer_and_wine'|  130|
|  5.0|     u'full_bar'|   91|
|  5.0|          'none'|   89|
+-----+----------------+-----+
only showing top 5 rows



In [25]:
df1.groupBy('stars','attributes.WheelchairAccessible').count().sort(desc('stars'),desc('count')).show(5)


+-----+--------------------+-----+
|stars|WheelchairAccessible|count|
+-----+--------------------+-----+
|  5.0|                null|  723|
|  5.0|                True|  602|
|  5.0|               False|   87|
|  4.5|                null| 4003|
|  4.5|                True| 2842|
+-----+--------------------+-----+
only showing top 5 rows



In [26]:
df1.groupBy('stars','attributes.DogsAllowed').count().sort(desc('stars'),desc('count')).show(5)

+-----+-----------+-----+
|stars|DogsAllowed|count|
+-----+-----------+-----+
|  5.0|       null|  942|
|  5.0|       True|  271|
|  5.0|      False|  199|
|  4.5|       null| 4491|
|  4.5|      False| 1718|
+-----+-----------+-----+
only showing top 5 rows



In [27]:
#common attributes for most 5 star restaurants
from pyspark.sql.functions import asc, desc
df1.groupBy('stars','attributes.BusinessAcceptsCreditCards','attributes.RestaurantsReservations',\
           'attributes.WiFi','attributes.RestaurantsDelivery','attributes.RestaurantsTakeOut',\
          'attributes.BusinessParking','attributes.RestaurantsGoodForGroups',\
          'attributes.HasTV','attributes.RestaurantsPriceRange2',\
          'attributes.GoodForMeal','attributes.OutdoorSeating').count().sort(desc('stars'),desc('count'))\
.filter((col("BusinessAcceptsCreditCards")!="null") & (col("RestaurantsReservations")!="null")\
       & (col("WiFi")!="null")&(col("RestaurantsTakeOut")!="null")\
       & (col("RestaurantsDelivery")!="null")&(col("BusinessParking")!="null")\
       & (col("RestaurantsGoodForGroups")!="null")&(col("HasTV")!="null")\
       & (col("RestaurantsPriceRange2")!="null")&(col("GoodForMeal")!="null")\
       & (col("OutdoorSeating")!="null")).show(5)

+-----+--------------------------+-----------------------+-------+-------------------+------------------+--------------------+------------------------+-----+----------------------+--------------------+--------------+-----+
|stars|BusinessAcceptsCreditCards|RestaurantsReservations|   WiFi|RestaurantsDelivery|RestaurantsTakeOut|     BusinessParking|RestaurantsGoodForGroups|HasTV|RestaurantsPriceRange2|         GoodForMeal|OutdoorSeating|count|
+-----+--------------------------+-----------------------+-------+-------------------+------------------+--------------------+------------------------+-----+----------------------+--------------------+--------------+-----+
|  5.0|                      True|                  False|  u'no'|              False|              True|{'garage': False,...|                    True|False|                     1|{'dessert': False...|          True|    3|
|  5.0|                      True|                  False|u'free'|               True|              True|{'g

In [ ]:
#https://observablehq.com/d/756aaa88cd766c30?ui=classic
#the link above is for top rated restaurant common characteristics graph visualization

In [28]:
# We can compare review count(rating count) for each states
from pyspark.sql.functions import asc, desc
df1.groupBy('state').sum('review_count')\
   .withColumnRenamed('sum(review_count)', 'total_review_count')\
   .sort(desc('total_review_count')).show()

+-----+------------------+
|state|total_review_count|
+-----+------------------+
|   MA|           1321763|
|   OR|            886301|
|   TX|            824359|
|   GA|            767356|
|   FL|            739760|
|   BC|            412818|
|   OH|            287914|
|   CO|             87300|
|   WA|             67453|
|   KS|                28|
|   NH|                17|
|  ABE|                14|
|   KY|                11|
|   WY|                 9|
|   VA|                 7|
|   MN|                 6|
+-----+------------------+



In [29]:
#count by categories, we see Mexican, Pizza ,cafe are top 3 popular categories
df1.groupBy('stars','categories').count().sort(desc('stars'),desc('count')).show(20)

+-----+--------------------+-----+
|stars|          categories|count|
+-----+--------------------+-----+
|  5.0|Restaurants, Mexican|   10|
|  5.0|Mexican, Restaurants|    8|
|  5.0|  Restaurants, Pizza|    8|
|  5.0|  Cafes, Restaurants|    5|
|  5.0|  Pizza, Restaurants|    5|
|  5.0| Indian, Restaurants|    5|
|  5.0|Restaurants, Food...|    5|
|  5.0|Middle Eastern, R...|    4|
|  5.0|Restaurants, Sand...|    4|
|  5.0|Restaurants, Amer...|    4|
|  5.0|         Restaurants|    4|
|  5.0|Restaurants, Viet...|    3|
|  5.0|Restaurants, Food...|    3|
|  5.0|Food, Coffee & Te...|    3|
|  5.0|Restaurants, Mexi...|    3|
|  5.0|Italian, Restaura...|    3|
|  5.0|Food Stands, Rest...|    3|
|  5.0|Food, Food Trucks...|    3|
|  5.0|Restaurants, Lati...|    3|
|  5.0|   Thai, Restaurants|    2|
+-----+--------------------+-----+
only showing top 20 rows

